In [1]:
import pandas as pd

In [2]:
brfss = pd.read_csv("./temp/edit_brfss.csv")
checkup = pd.read_csv("./temp/건강검진.csv", encoding="cp949")
regionheart = pd.read_csv("./temp/지역별 심장질환.csv", encoding="cp949")
medical = pd.read_csv("./temp/hos_all.csv")

In [3]:
def check_dataframe(df):
    print(df.head())
    print(df.info())
    
    for col in df.columns:
        if len(df[col].unique()) <= 10:
            print(df[col].value_counts())
            print("*"*20)

## brfss
1. CHOLESTEROL, LIVER1, LIVER2 열 제거
2. SMOKE를 제외한 열 중 하나라도 결측값이 있는 행 제거
3. HEIGHT, WEIGHT 열 5 단위로 변경

In [4]:
check_dataframe(brfss)

   HEARTDISEASE SEX  HEIGHT     WEIGHT  BLIND  DEAF  DIABETE  CHOLESTEROL  \
0           0.0   F  149.86  32.658651    0.0   0.0      0.0          1.0   
1           0.0   F  167.64        NaN    1.0   0.0      1.0          1.0   
2           0.0   F  165.10  77.110703    0.0   0.0      1.0          2.0   
3           0.0   F  162.56  88.450512    0.0   0.0      1.0          1.0   
4           1.0   M  180.34  93.440028    0.0   1.0      1.0          1.0   

   HYPERTENSION  KIDNEY  LIVER1  LIVER2  ALCOHOL  SMOKE  
0           0.0     0.0     NaN     NaN      0.0    1.0  
1           1.0     1.0     NaN     NaN      0.0    0.0  
2           1.0     0.0     NaN     NaN      0.0    0.0  
3           1.0     0.0     NaN     NaN      1.0    0.0  
4           0.0     0.0     NaN     NaN      0.0    0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438693 entries, 0 to 438692
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        -------------- 

### 열 제거
- CHOLESTEROL, LIVER 제거

In [5]:
brfss.drop(["CHOLESTEROL", "LIVER1", "LIVER2"], axis=1, inplace=True)

### 결측치 처리
- SMOKE 제외한 열 중에서 하나라도 결측치가 있는 행 제거

In [6]:
brfss.isnull().sum()

HEARTDISEASE     2655
SEX                 0
HEIGHT          24559
WEIGHT          40575
BLIND           17351
DEAF            16398
DIABETE           985
HYPERTENSION     1912
KIDNEY           1813
ALCOHOL         30589
SMOKE           24461
dtype: int64

In [7]:
brfss.isnull().sum() / len(brfss)

HEARTDISEASE    0.006052
SEX             0.000000
HEIGHT          0.055982
WEIGHT          0.092491
BLIND           0.039552
DEAF            0.037379
DIABETE         0.002245
HYPERTENSION    0.004358
KIDNEY          0.004133
ALCOHOL         0.069728
SMOKE           0.055759
dtype: float64

In [8]:
col_list = list(brfss.columns)
col_list.remove("SMOKE")

brfss.dropna(how="any", subset=col_list, inplace=True)

In [9]:
brfss["SMOKE"].fillna(9, inplace=True)

### HEIGHT, WEIGHT
- 검진자의 키(5CM 단위)
    - 예) 100-104CM --> 100CM
- 검진자의 몸무게(5KG 단위)
    - 예) 25~29KG -> 25KG

In [10]:
brfss["HEIGHT"] = brfss["HEIGHT"] // 5 * 5

In [11]:
brfss["WEIGHT"] = brfss["WEIGHT"] // 5 * 5

### 확인 및 저장

In [12]:
check_dataframe(brfss)

   HEARTDISEASE SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  \
0           0.0   F   145.0    30.0    0.0   0.0      0.0           0.0   
2           0.0   F   165.0    75.0    0.0   0.0      1.0           1.0   
3           0.0   F   160.0    85.0    0.0   0.0      1.0           1.0   
4           1.0   M   180.0    90.0    0.0   1.0      1.0           0.0   
5           0.0   M   190.0    85.0    0.0   0.0      0.0           0.0   

   KIDNEY  ALCOHOL  SMOKE  
0     0.0      0.0    1.0  
2     0.0      0.0    0.0  
3     0.0      1.0    0.0  
4     0.0      0.0    0.0  
5     0.0      0.0    1.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 373951 entries, 0 to 438692
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   HEARTDISEASE  373951 non-null  float64
 1   SEX           373951 non-null  object 
 2   HEIGHT        373951 non-null  float64
 3   WEIGHT        373951 non-null  float64
 4   B

In [13]:
brfss.to_csv("../data/brfss.csv", index=False)

## 건강검진
1. LIVER 열 제거
2. SEX 열 F, M으로 변경
3. 지역 열 긴 이름으로 변경(서울 -> 서울특별시)
4. encoding 변경(cp949 -> utf-8)

In [14]:
check_dataframe(checkup)

   지역  SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  LIVER  \
0  세종    1     165      60      0     0        0             0       0      0   
1  대구    2     150      65      0     0        0             0       1      0   
2  서울    2     155      55      0     0        0             0       1      0   
3  울산    1     160      70      0     1        1             0       0      1   
4  경기    2     155      50      0     0        0             1       1      0   

   ALCOHOL  SMOKE  
0      0.0      0  
1      0.0      0  
2      0.0      0  
3      0.0      0  
4      0.0      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           int64  
 2   HEIGHT        int64  
 3   WEIGHT        int64  
 4   BLIND         int64  
 5   DEAF          int64  
 6   DIABETE       int64  
 7   HYPERTENSION  int64  
 8   KIDNE

In [15]:
# LIVER 제거
checkup.drop(["LIVER"], axis=1, inplace=True)

In [16]:
region = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}

checkup["SEX"].replace({1:"M", 2:"F"}, inplace=True)
checkup["지역"].replace(region, inplace=True)

In [17]:
checkup.isnull().sum()

지역              0
SEX             0
HEIGHT          0
WEIGHT          0
BLIND           0
DEAF            0
DIABETE         0
HYPERTENSION    0
KIDNEY          0
ALCOHOL         0
SMOKE           0
dtype: int64

In [18]:
check_dataframe(checkup)

        지역 SEX  HEIGHT  WEIGHT  BLIND  DEAF  DIABETE  HYPERTENSION  KIDNEY  \
0  세종특별자치시   M     165      60      0     0        0             0       0   
1    대구광역시   F     150      65      0     0        0             0       1   
2    서울특별시   F     155      55      0     0        0             0       1   
3    울산광역시   M     160      70      0     1        1             0       0   
4      경기도   F     155      50      0     0        0             1       1   

   ALCOHOL  SMOKE  
0      0.0      0  
1      0.0      0  
2      0.0      0  
3      0.0      0  
4      0.0      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039244 entries, 0 to 2039243
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   지역            object 
 1   SEX           object 
 2   HEIGHT        int64  
 3   WEIGHT        int64  
 4   BLIND         int64  
 5   DEAF          int64  
 6   DIABETE       int64  
 7   HYPERTENSION  int64  
 8   KIDNEY        int64  
 

In [19]:
checkup.to_csv("../data/checkup.csv", index=False, encoding="utf-8")

## 지역별 심장환자 현황
1. encoding 변경(cp949 -> utf-8)

In [20]:
check_dataframe(regionheart)

      지역  심장질환사망자  심장장애      주민등록인구
0    강원도   4900.0   121   1538492.0
1    경기도  25708.0  1295  13565450.0
2   경상남도  10312.0   268   3314183.0
3   경상북도   9880.0   311   2626609.0
4  광주광역시   2712.0   140   1441611.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역       17 non-null     object 
 1   심장질환사망자  17 non-null     float64
 2   심장장애     17 non-null     int64  
 3   주민등록인구   17 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 672.0+ bytes
None


In [21]:
regionheart.isnull().sum()

지역         0
심장질환사망자    0
심장장애       0
주민등록인구     0
dtype: int64

In [22]:
regionheart.columns

Index(['지역', '심장질환사망자', '심장장애', '주민등록인구'], dtype='object')

In [23]:
regionheart = regionheart[['지역', '주민등록인구', '심장질환사망자', '심장장애']]

In [24]:
regionheart.to_csv("./temp/region_heart.csv", index=False, encoding="utf-8")

## 의료수준
1. 첫 번째 열 삭제
2. 일반의, 전문의, 간호사, 임상병리사 열 데이터 타입 변경(object -> int)
3. 지역 열 긴 이름으로 변경

In [25]:
check_dataframe(medical)

   Unnamed: 0  지역  상급종합병원  종합병원   병원  심장혈관흉부외과  심장혈관흉부외과.1    일반의     전문의  \
0           0  서울      14    42  222      57.0         305  1,287  25,109   
1           1  부산       3    25  136      10.0          83    377   6,966   
2           2  인천       3    17   56       8.0          55    260   4,433   
3           3  대구       5    13   88       1.0          51    184   5,060   
4           4  광주       2    21   84       3.0          33    148   3,092   

      간호사  임상병리사  심장혈관흉부외과.2  심장혈관흉부외과.3   일반병실   일반병상  중환병실  중환병상  구급차 (대)  \
0  58,475  6,831        57.0         305  13846  45730   250  2660      177   
1  19,705  1,982        10.0          83   7646  24605    64   860       70   
2  12,551  1,343         8.0          55   3249  12248    47   610       55   
3  13,948  1,578         1.0          51   4580  16622    60   584       72   
4   9,414    941         3.0          33   4427  13894    27   408       30   

   구급대원수  
0   1473  
1    633  
2    495  
3    592  
4    27

In [26]:
# 첫 번째 열 삭제
medical = medical.iloc[:, 1:]

In [27]:
# 데이터 타입 변환
def change_to_int(x):
    return x.str.replace(",","").replace("-", "0").astype("int64")

medical[["일반의", "전문의", "간호사", "임상병리사"]] = medical[["일반의", "전문의", "간호사", "임상병리사"]].apply(change_to_int, axis=1)

In [28]:
# 지역 이름 변환
region = {"강원": "강원도", "경기": "경기도", "경남": "경상남도", "경북": "경상북도", 
          "전남": "전라남도", "전북": "전라북도", "충남": "충청남도", "충북": "충청북도", 
          "광주": "광주광역시", "대구": "대구광역시", "대전": "대전광역시", "부산": "부산광역시", 
          "서울": "서울특별시", "울산": "울산광역시", "인천": "인천광역시", 
          "세종": "세종특별자치시", "제주": "제주특별자치도"}

medical["지역"].replace(region, inplace=True)

In [29]:
medical.dtypes

지역             object
상급종합병원          int64
종합병원            int64
병원              int64
심장혈관흉부외과      float64
심장혈관흉부외과.1      int64
일반의             int64
전문의             int64
간호사             int64
임상병리사           int64
심장혈관흉부외과.2    float64
심장혈관흉부외과.3      int64
일반병실            int64
일반병상            int64
중환병실            int64
중환병상            int64
구급차 (대)         int64
구급대원수           int64
dtype: object

### 심장혈관흉부외과

In [30]:
medical[["심장혈관흉부외과", "심장혈관흉부외과.1", "심장혈관흉부외과.2", "심장혈관흉부외과.3"]]
# 심장혈관흉부외과, 심장혈관흉부외과.2 동일
# 심장혈관흉부외과.1, 심장혈관흉부외과.3 동일

,심장혈관흉부외과,심장혈관흉부외과.1,심장혈관흉부외과.2,심장혈관흉부외과.3
0,57.0,305,57.0,305
1,10.0,83,10.0,83
2,8.0,55,8.0,55
3,1.0,51,1.0,51
4,3.0,33,3.0,33
5,4.0,50,4.0,50
6,5.0,21,5.0,21
7,9.0,209,9.0,209
8,1.0,36,1.0,36
9,4.0,36,4.0,36


In [31]:
medical.fillna(0, inplace=True)
medical.drop(["심장혈관흉부외과.2", "심장혈관흉부외과.3"], axis=1, inplace=True)

In [32]:
medical.columns

Index(['지역', '상급종합병원', '종합병원', '병원', '심장혈관흉부외과', '심장혈관흉부외과.1', '일반의', '전문의',
       '간호사', '임상병리사', '일반병실', '일반병상', '중환병실', '중환병상', '구급차 (대)', '구급대원수'],
      dtype='object')

In [33]:
medical.rename({"심장혈관흉부외과": "심장혈관흉부외과의원", "심장혈관흉부외과.1":"심장혈관흉부외과전문의"}, axis=1, inplace=True)

### 구급대 현황 다시 

In [34]:
df = pd.read_excel("./rawdata/119_구급대_현황_20221220173712.xls", skiprows=1)

In [35]:
temp = df[["본부별(1)", "운영구급차 (대)", "구급대원 (명)"]].drop([0, 1, 2, 21, 22], axis=0)

# 타입 변환
temp[["운영구급차 (대)", "구급대원 (명)"]] = temp[["운영구급차 (대)", "구급대원 (명)"]].astype("int64")

# 컬럼명 변환
temp.columns = ["지역", "구급차", "구급대원"]

# reset index
temp.reset_index(drop=True, inplace=True)

# 창원 + 경남 -> 경남
temp.iloc[15, 1:] = temp.iloc[15, 1:] + temp.iloc[17, 1:]
temp.drop([17], axis=0, inplace=True)

# 제주도 -> 제주특별자치도
temp["지역"].replace({"제주도": "제주특별자치도"}, inplace=True)

temp

,지역,구급차,구급대원
0,서울특별시,177,1473
1,부산광역시,70,633
2,대구광역시,55,495
3,인천광역시,72,592
4,광주광역시,30,270
5,대전광역시,34,286
6,울산광역시,31,261
7,세종특별자치시,11,99
8,경기도,261,1896
9,강원도,128,1014


In [36]:
medical.drop(['구급차 (대)', '구급대원수'], axis=1, inplace=True)

In [37]:
medical = pd.merge(left=medical, right=temp, on="지역")

### 확인 및 저장

In [38]:
check_dataframe(medical)

      지역  상급종합병원  종합병원   병원  심장혈관흉부외과의원  심장혈관흉부외과전문의   일반의    전문의    간호사  \
0  서울특별시      14    42  222        57.0          305  1287  25109  58475   
1  부산광역시       3    25  136        10.0           83   377   6966  19705   
2  인천광역시       3    17   56         8.0           55   260   4433  12551   
3  대구광역시       5    13   88         1.0           51   184   5060  13948   
4  광주광역시       2    21   84         3.0           33   148   3092   9414   

   임상병리사   일반병실   일반병상  중환병실  중환병상  구급차  구급대원  
0   6831  13846  45730   250  2660  177  1473  
1   1982   7646  24605    64   860   70   633  
2   1343   3249  12248    47   610   72   592  
3   1578   4580  16622    60   584   55   495  
4    941   4427  13894    27   408   30   270  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지역           17 non-null     object 
 1   상급종합병원       17 non-n

In [39]:
medical.to_csv("./temp/medical.csv", index=False, encoding="utf-8")

## 지역별 심장질환 + 의료수준

In [40]:
region = pd.merge(left=regionheart, right=medical, on="지역")

In [41]:
check_dataframe(region)

      지역      주민등록인구  심장질환사망자  심장장애  상급종합병원  종합병원   병원  심장혈관흉부외과의원  \
0    강원도   1538492.0   4900.0   121       2    13   37         1.0   
1    경기도  13565450.0  25708.0  1295       5    62  277         9.0   
2   경상남도   3314183.0  10312.0   268       3    23  134         2.0   
3   경상북도   2626609.0   9880.0   311       0    20   55         2.0   
4  광주광역시   1441611.0   2712.0   140       2    21   84         3.0   

   심장혈관흉부외과전문의   일반의    전문의    간호사  임상병리사   일반병실   일반병상  중환병실  중환병상  구급차  구급대원  
0           36   220   2211   6622    661   2398   8403    53   438  128  1014  
1          209  1039  19686  46768   5526  13890  48092   226  2209  261  1896  
2           72   420   4856  14959   1829   6636  22556    50   555  139  1194  
3           51   404   3167  10143    988   7635  32155    49   495  146  1279  
4           33   148   3092   9414    941   4427  13894    27   408   30   270  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 19 

In [42]:
region.to_csv("../data/region.csv", index=False, encoding="utf-8")